In [22]:
import nltk

nltk.download('punkt')  # Punkt is a  Sentence Tokenizer
nltk.download('wordnet') #corpus / lexical database
from nltk.stem import WordNetLemmatizer  # for lemmatization

lemmatizer = WordNetLemmatizer()
import json   
import pickle  
import random
import tkinter  
from tkinter import *

import numpy as np  
from keras.layers import Activation, Dense, Dropout

from keras.models import Sequential, load_model
from keras.optimizers import SGD

[nltk_data] Downloading package punkt to /home/pradeep/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pradeep/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
words=[] 
classes = [] 
documents = [] 
useless_words = ['?', '!']  
data = open('intents.json').read() 
intents = json.loads(data)


In [24]:
intents 

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'tasks',
   'patterns': ['What can you do?',
    'What are your features?',
    'What are you abilities',
    'can you sing',
    'can you talk'],
   'responses': ['I can do whatever you asks me to do',
    'I can talk and do things for you',
    "Right now i'm in developing stage as soon i'm developed, I can do everything"]},
  {'tag': 'talk',
   'patterns': ['can you sing', 'can you talk', 'can you speak'],
  

In [25]:
          

for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        word = nltk.word_tokenize(pattern) 
        words.extend(word) 
        documents.append((word, intent['tag']))  

        
        if intent['tag'] not in classes: 
            classes.append(intent['tag'])

In [26]:

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in useless_words] 
words = sorted(list(set(words))) 

classes = sorted(list(set(classes)))  

pickle.dump(classes,open('classes.pkl','wb'))  
pickle.dump(words,open('words.pkl','wb')) 

In [17]:
training_data = []
output_empty = [0] * len(classes)
for document in documents:
    bag = [] 
    pattern_words = document[0] 
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words] 
    for w in words: 
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1 

    training_data.append([bag, output_row])
random.shuffle(training_data) 
training_data = np.array(training_data)
train_x = list(training_data[:,0])
train_y = list(training_data[:,1])




/tmp/ipykernel_24025/366098736.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_data = np.array(training_data)


In [27]:

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu')) #128 neurons,
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu')) #64 neurons
model.add(Dropout(0.5)) 
model.add(Dense(len(train_y[0]), activation='softmax'))  

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])  

history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1) 

model.save('chatbot_model.h5', history) #save the model


Epoch 1/200
16/16 [==============================] - 0s 1ms/step - loss: 2.9825 - accuracy: 0.0385
Epoch 2/200
16/16 [==============================] - 0s 1ms/step - loss: 2.9312 - accuracy: 0.0897
Epoch 3/200
16/16 [==============================] - 0s 1ms/step - loss: 2.7951 - accuracy: 0.1154
Epoch 4/200
16/16 [==============================] - 0s 1ms/step - loss: 2.7523 - accuracy: 0.1538
Epoch 5/200
16/16 [==============================] - 0s 1ms/step - loss: 2.6896 - accuracy: 0.2051
Epoch 6/200
16/16 [==============================] - 0s 1ms/step - loss: 2.5362 - accuracy: 0.2949
Epoch 7/200
16/16 [==============================] - 0s 1ms/step - loss: 2.4704 - accuracy: 0.2051
Epoch 8/200
16/16 [==============================] - 0s 2ms/step - loss: 2.4195 - accuracy: 0.2436
Epoch 9/200
16/16 [==============================] - 0s 1ms/step - loss: 2.3240 - accuracy: 0.2949
Epoch 10/200
16/16 [==============================] - 0s 2ms/step - loss: 2.2281 - accuracy: 0.3205
Epoch 11/

In [28]:
 
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [29]:

def clean(sentence): 
    words_sentence = nltk.word_tokenize(sentence)
    words_sentence = [lemmatizer.lemmatize(word.lower()) for word in words_sentence]
    return words_sentence



def bag_words(sentence, words):
    
    words_sentence = clean(sentence) 
    bag = [0]*len(words) 
    
    for ws in words_sentence:
        for i,w in enumerate(words):
            if w == ws:
                bag[i] = 1 
                
    return(np.array(bag))

def prediction(sentence, model):
    
    p = bag_words(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def final_response(classes_predicted, all_intents):
    tag = classes_predicted[0]['intent'] 
    list_intents = all_intents['intents']
    for i in list_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])  
            break
    return result

def chatbot_output(message):
    classes_predicted = prediction(message, model) 
    res = final_response(classes_predicted, intents) 
    return res




In [30]:
#GUI with tkinter

def send():
    message = box.get("1.0",'end-1c').strip()
    box.delete("0.0",END)

    if message != '':
        chat.config(state=NORMAL)
        chat.insert(END, "You: " + message + '\n\n')
        chat.config(foreground="#442065", font=("Verdana", 12 ))

        res = chatbot_output(message)
        chat.insert(END, "Bot: " + res + '\n\n')

        chat.config(state=DISABLED)
        chat.yview(END)


base = Tk()
base.title("Mariem BOT")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
chat = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

chat.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=chat.yview, cursor="heart")
chat['yscrollcommand'] = scrollbar.set

#Create Button to send message
button = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#18809F", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
box = Text(base, bd=0, bg="#96DCF0",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
chat.place(x=6,y=6, height=386, width=370)
box.place(x=128, y=401, height=90, width=265)
button.place(x=6, y=401, height=90)

base.mainloop()


1/1 [==============================] - 0s 10ms/step
